In [1]:
import pandas as pd
import numpy as np# Import library
import pandas as pd
from tqdm import tqdm
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import csv

In [2]:
loan_user = pd.read_csv('loanuser_fin.csv')

In [3]:
loan_null = loan_user.loc[loan_user['is_applied'].isnull()]

In [4]:
loan_user = loan_user.loc[loan_user['is_applied'].notnull()]

In [5]:
loan_user

,loan_limit,loan_rate,gender,credit_score,yearly_income,desired_amount,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,work_day,Age,label_purpose,label_employment_type,label_houseown_type,label_income_type,is_applied
9818,11.0,16.5,1.0,8,72.0,20.0,0.0,0.0,7.0,106.0,34,45,3,3,3,0,0.0
9819,3.0,20.0,1.0,8,72.0,20.0,0.0,0.0,7.0,106.0,34,45,3,3,3,0,0.0
9820,10.0,13.5,1.0,6,39.0,80.0,0.0,0.0,2.0,58.0,124,39,1,3,3,0,0.0
9821,22.0,15.9,1.0,6,39.0,80.0,0.0,0.0,2.0,58.0,124,39,1,3,3,0,0.0
9822,10.0,18.4,1.0,6,39.0,80.0,0.0,0.0,2.0,58.0,124,39,1,3,3,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9965016,18.0,7.0,0.0,7,35.0,6.0,0.0,0.0,3.0,27.0,9,45,3,1,2,1,0.0
9965017,18.0,7.2,0.0,7,35.0,6.0,0.0,0.0,3.0,27.0,9,45,3,1,2,1,0.0
9965018,16.0,9.9,0.0,7,35.0,6.0,0.0,0.0,3.0,27.0,9,45,3,1,2,1,0.0
9965019,30.0,15.1,0.0,7,35.0,6.0,0.0,0.0,3.0,27.0,9,45,3,1,2,1,0.0


In [6]:
loan_user['is_applied'].value_counts()

0.0    7180504
1.0     414667
Name: is_applied, dtype: int64

In [7]:
loan_user.columns

Index(['loan_limit', 'loan_rate', 'gender', 'credit_score', 'yearly_income',
       'desired_amount', 'personal_rehabilitation_yn',
       'personal_rehabilitation_complete_yn', 'existing_loan_cnt',
       'existing_loan_amt', 'work_day', 'Age', 'label_purpose',
       'label_employment_type', 'label_houseown_type', 'label_income_type',
       'is_applied'],
      dtype='object')

#### 언더샘플링

In [8]:
X = loan_user[['loan_limit', 'loan_rate', 'gender', 'credit_score', 'yearly_income',
       'desired_amount', 'personal_rehabilitation_yn',
       'personal_rehabilitation_complete_yn', 'existing_loan_cnt',
       'existing_loan_amt', 'work_day', 'Age', 'label_purpose',
       'label_employment_type', 'label_houseown_type', 'label_income_type']]
X

,loan_limit,loan_rate,gender,credit_score,yearly_income,desired_amount,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,work_day,Age,label_purpose,label_employment_type,label_houseown_type,label_income_type
9818,11.0,16.5,1.0,8,72.0,20.0,0.0,0.0,7.0,106.0,34,45,3,3,3,0
9819,3.0,20.0,1.0,8,72.0,20.0,0.0,0.0,7.0,106.0,34,45,3,3,3,0
9820,10.0,13.5,1.0,6,39.0,80.0,0.0,0.0,2.0,58.0,124,39,1,3,3,0
9821,22.0,15.9,1.0,6,39.0,80.0,0.0,0.0,2.0,58.0,124,39,1,3,3,0
9822,10.0,18.4,1.0,6,39.0,80.0,0.0,0.0,2.0,58.0,124,39,1,3,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9965016,18.0,7.0,0.0,7,35.0,6.0,0.0,0.0,3.0,27.0,9,45,3,1,2,1
9965017,18.0,7.2,0.0,7,35.0,6.0,0.0,0.0,3.0,27.0,9,45,3,1,2,1
9965018,16.0,9.9,0.0,7,35.0,6.0,0.0,0.0,3.0,27.0,9,45,3,1,2,1
9965019,30.0,15.1,0.0,7,35.0,6.0,0.0,0.0,3.0,27.0,9,45,3,1,2,1


In [9]:
y = loan_user['is_applied']

In [ ]:
# 언더샘플링
from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy='majority')
X_under, y_under = undersample.fit_resample(X, y)

In [ ]:
y_under.value_counts()

In [10]:
# 기본 샘플링 train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(loan_user[['loan_limit', 'loan_rate', 'gender', 'credit_score', 'yearly_income',
       'desired_amount', 'personal_rehabilitation_yn',
       'personal_rehabilitation_complete_yn', 'existing_loan_cnt',
       'existing_loan_amt', 'work_day', 'Age', 'label_purpose',
       'label_employment_type', 'label_houseown_type', 'label_income_type', ]],loan_user['is_appislied'], test_size=0.2, random_state=42 )

In [ ]:
# 언더샘플링 train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_under,y_under, test_size=0.2, random_state=42 )

In [11]:
print('학습 데이터 레이블 값 비율')
print(y_train.value_counts()/y_train.shape[0] * 100)
print('테스트 데이터 레이블 값 비율')
print(y_test.value_counts()/y_test.shape[0] * 100)


학습 데이터 레이블 값 비율
0.0    94.543901
1.0     5.456099
Name: is_applied, dtype: float64
테스트 데이터 레이블 값 비율
0.0    94.526328
1.0     5.473672
Name: is_applied, dtype: float64


In [12]:
train = pd.concat([X_train, y_train], axis= 0)

In [16]:
from sklearn.utils.class_weight import compute_class_weight

# 불균형한 클래스인 것으로 확인되어 class_weight = 'balanced'로 설정해준다. 
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weights = dict(zip(classes, weights))

In [17]:
from catboost import CatBoostClassifier

In [18]:
from sklearn.model_selection import cross_val_score
cb = CatBoostClassifier(learning_rate= 0.03, max_depth= 10, n_estimators= 100, 
                    class_weights= class_weights, subsample = 0.8, colsample_bylevel=1.0, random_state=42, verbose =0)

# 성능 지표는 정확도(accuracy) , 교차 검증 세트는 5개 
scores = cross_val_score(cb , X, y, scoring='accuracy',cv=5)

print('교차 검증별 정확도:',np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores),4))

교차 검증별 정확도: [0.7223 0.7259 0.7264 0.7265 0.7202]
평균 검증 정확도: 0.7243


In [19]:
cb.fit(X_train, y_train)
y_pred = cb.predict(X_test)

In [20]:
from sklearn import metrics

In [21]:
print('훈련세트 정확도: {:.3f}' .format(cb.score(X_train, y_train)))
print('테스트세트 정확도: {:.3f}' .format(cb.score(X_test, y_test)))
print('f1 score 1', metrics.f1_score(y_test, y_pred))

훈련세트 정확도: 0.725
테스트세트 정확도: 0.726
f1 score 1 0.239537849855943
